In [1]:
cd ../src

c:\Users\fredl\Documents\Cours\M1 Androide\S2\ML\Projet\src


C:\Users\fredl\AppData\Roaming\Python\Python313\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from activation_func import *
from loss import *
from mltools import *
from module import *
from optimizers import *
from mltools import *

In [ ]:
import time
import tracemalloc

# Hypothèse : X_train, y_train, X_test, y_test sont déjà préparés

model = Sequential(
    Linear(X_train.shape[1], 64),
    ReLU(),
    Linear(64, 32),
    ReLU(),
    Linear(32, np.unique(y_train).size),  # nombre de classes
    Softmax()
)
loss_fn = CrossEntropyLoss()
optimizer = Optim(model, loss_fn, 0.01)

batch_sizes = [1, 16, 64, 256]
results = {}

for batch_size in batch_sizes:
    epoch_precisions = []
    epoch_times = []
    epoch_memories = []
    n_epochs = 20

    for epoch in range(n_epochs):
        start_time = time.time()
        tracemalloc.start()
        
        # Shuffle and batch training
        for i in range(0, len(X_train), batch_size):
            X_batch = X_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]
            model.train_on_batch(X_batch, y_batch)
        
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        elapsed = time.time() - start_time

        # Évaluation sur le jeu de test
        y_pred = model.predict(X_test)
        precision = np.mean(y_pred == y_test)

        epoch_precisions.append(precision)
        epoch_times.append(elapsed)
        epoch_memories.append(peak / 1024 / 1024)  # en Mo

    results[batch_size] = {
        "precisions": epoch_precisions,
        "times": epoch_times,
        "memories": epoch_memories
    }

# Rapport visuel : précision en fonction du temps cumulé pour chaque batch_size
for batch_size in batch_sizes:
    plt.plot(np.cumsum(results[batch_size]["times"]), results[batch_size]["precisions"], label=f"Batch {batch_size}")

plt.xlabel("Temps cumulé (s)")
plt.ylabel("Précision")
plt.title("Évolution de la précision en fonction du temps réel pour chaque taille de batch")
plt.legend()
plt.show()